In [1]:
import torch
from torch import nn
from torch import optim

# A Toy Dataset
data = torch.tensor([[0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,1,0],[1,1,0,0],[1,0,1,0],[0,1,1,0],[1,1,1,0],[0,0,0,1],[1,0,0,1],[0,1,0,1],[0,0,1,1],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1.]])
target = torch.tensor([[0],[0],[0],[0],[0],[0],[0],[0],[1],[1],[1],[1],[1],[1],[1],[1.]])


# A Toy Model
model = nn.Sequential(
        nn.Linear(4, 3),
        nn.Tanh(),
        nn.Linear(3, 1),
        nn.Sigmoid()
    )

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(3.5702)
tensor(3.4189)
tensor(3.2462)
tensor(3.0390)
tensor(2.7925)
tensor(2.5109)
tensor(2.2076)
tensor(1.9019)
tensor(1.6136)
tensor(1.3571)
tensor(1.1390)
tensor(0.9591)
tensor(0.8130)
tensor(0.6952)
tensor(0.6001)
tensor(0.5230)
tensor(0.4599)
tensor(0.4080)
tensor(0.3648)
tensor(0.3285)


In [2]:

import syft as sy
import time
hook = sy.TorchHook(torch)

# from torchviz import make_dot, make_dot_from_trace
from torch.autograd import Variable

# A Toy Dataset
x = torch.tensor([[0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,1,0],[1,1,0,0],[1,0,1,0],[0,1,1,0],[1,1,1,0],[0,0,0,1],[1,0,0,1],[0,1,0,1],[0,0,1,1],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1.]])
y = torch.tensor([[0],[0],[0],[0],[0],[0],[0],[0],[1],[1],[1],[1],[1],[1],[1],[1.]])

#   Variables for performance metrics
start_time = time.time()
epochs = 300
lr = 0.2
counter = 0


# Define 2 chained models
models = [
    nn.Sequential(
        nn.Linear(4, 3),
        nn.Tanh()
    ),
    nn.Sequential(
        nn.Linear(3, 1),
        nn.Sigmoid()
    )
]

# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(params=model.parameters(),lr=lr)
    for model in models
]

# create some workers
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
workers = alice, bob

# Send Model Segments to starting locations
model_locations = [alice, bob]

for model, location in zip(models, model_locations):
    model.send(location)

# Create a remote copy of the dataset for each worker
datasets = [
    sy.BaseDataset(x.send(alice), y.send(bob))
    for worker in workers
]

/Users/adamhall/anaconda3/envs/pysyft/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/adamhall/anaconda3/envs/pysyft/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/adamhall/anaconda3/envs/pysyft/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/adamhall/anaconda3/envs/pysyft

In [3]:
def forward(models, x):

    inputs = []
    outputs = []
    
    # First: provide x as input
    inputs.append(x)
    outputs.append(models[0](inputs[-1]))
    
    # MOve a copy of the inputs from the previous layer to the layer in front
    inputs.append(outputs[-1].copy().move(bob))
    outputs.append(models[1](inputs[-1]))
    
    return inputs, outputs

In [4]:
def backward(models, optimizers, inputs, outputs, dataset):
    data, targets = dataset.data, dataset.targets
        
    # Destroy pre-existing gradient of final layer
    optimizers[-1].zero_grad()
    loss = (((outputs[-1] - targets)**2).sum())
    loss.backward()
    # End layer sends the gradient of the activation signal back to the layer behind
    input_gradient = inputs[-1].grad.clone().move(alice)
    # End layer updates weights
    optimizers[1].step()

    # Compute Final Layer, same but now input is the real input data
    optimizers[0].zero_grad()
    segment_output = outputs[0]
    # Dot join the gradient of the input to the layer in front to the output of this segment
    intermediate_loss = torch.matmul(torch.t(segment_output), input_gradient).sum()
    intermediate_loss.backward()
    optimizers[0].step()
        
    return segment_output[-1], loss

In [5]:
for i in range(20):
    inputs, outputs = forward(models, datasets[0].data)
    prediction, loss = backward(models, optimizers, inputs, outputs, datasets[0])
    print(loss.get())

tensor(4.3027, requires_grad=True)
tensor(4.0265, requires_grad=True)
tensor(3.8157, requires_grad=True)
tensor(3.6322, requires_grad=True)
tensor(3.4653, requires_grad=True)
tensor(3.3123, requires_grad=True)
tensor(3.1722, requires_grad=True)
tensor(3.0442, requires_grad=True)
tensor(2.9272, requires_grad=True)
tensor(2.8203, requires_grad=True)
tensor(2.7225, requires_grad=True)
tensor(2.6330, requires_grad=True)
tensor(2.5509, requires_grad=True)
tensor(2.4754, requires_grad=True)
tensor(2.4058, requires_grad=True)
tensor(2.3416, requires_grad=True)
tensor(2.2822, requires_grad=True)
tensor(2.2270, requires_grad=True)
tensor(2.1757, requires_grad=True)
tensor(2.1279, requires_grad=True)


#### I am looking train this model while it is split across two hosts. I attempt to transfer loss backward for the backpropogation by sending the gradients to join to the output of the previous layer. This works but not as well as the other model which is centralised. I am wondering how i can improve this loss transfer formula.